# Load Packages

In [1]:
import numpy as np
import scipy.special
import numba

import matplotlib.pyplot as plt
%matplotlib inline

#!pip install memory_profiler
import memory_profiler
%load_ext memory_profiler

# Load Demo Data

In [2]:
with np.load('data/demo1.npz') as data:
    x = data['px'][:, 0]

# Modeling
Let $x_t$ a time series, 
$t\in\mathbb{N}$ the time step,
$\Delta^d$ the difference operator of fractional order $d\in\mathbb{R}^+$,
and $m$ the truncation order

**Formula 1** (see Schoffer, 2003, p.50; The formula will **fail** for integral order $d\in\mathbb{N}$)

$$
(\Delta^d x)_t = \sum_{k=0}^\infty \frac{\Gamma(k - d)}{\Gamma(k + 1) \, \Gamma(-d)} \, x_{t-k} \\
(\Delta^d x)_t \approx \sum_{k=0}^m \frac{\Gamma(k - d)}{\Gamma(k + 1) \, \Gamma(-d)} \, x_{t-k}
$$



**Formula 2** (see Balisarsingh, 2013, pp.9737-9742)

$$
(\Delta^d x)_t = \sum_{k=0}^\infty (-1)^k \frac{\Gamma(d + 1)}{k! \, \Gamma(d - k + 1)} \, x_{t-k} \\
(\Delta^d x)_t \approx \sum_{k=0}^m (-1)^k \frac{\Gamma(d + 1)}{k! \, \Gamma(d - k + 1)} \, x_{t-k}
$$


**Formula 3/4** (see Lopez, 2018, p.78, from the 'iterative estimation' formula; Jensen and Nielsen, 2014)

$$
(\Delta^d x)_t = x_t + \sum_{k=1}^\infty \left(\prod_{i=1}^k \frac{d - i + 1}{i} \right) x_{t-k} \\
(\Delta^d x)_t \approx x_t + \sum_{k=1}^m \left(\prod_{i=1}^k \frac{d - i + 1}{i} \right) x_{t-k}
$$

In [3]:
def frac_weights_1(d: float, m: int) -> np.ndarray:
    w = np.empty((m + 1,))
    for k in range(m + 1):
        w[k] = scipy.special.gamma(k - d) \
               / (scipy.special.gamma(k + 1) * scipy.special.gamma(-d))
    return w

In [4]:
def frac_weights_2(d: float, m: int) -> np.ndarray:
    w = np.empty((m + 1,))
    for k in range(m + 1):
        w[k] = np.power(-1, k) * scipy.special.gamma(d + 1) \
               / (scipy.special.factorial(k) * scipy.special.gamma(d - k + 1))
    return w

In [5]:
def frac_weights_3(d: float, m: int) -> np.ndarray:
    w = np.empty((m + 1,))
    w[0] = 1
    for j in range(1, m + 1):
        w[j] = -w[j - 1] * ((d - j + 1) / j)
    return w

In [6]:
def frac_weights_4(d: float, m: int) -> np.ndarray:
    w = [1]
    for k in range(1, m + 1):
        w.append(-w[-1] * ((d - k + 1) / k))
    return np.array(w)

In [7]:
@numba.jit(nopython=True)
def frac_weights_5(d: float, m: int) -> np.ndarray:
    w = [1]
    for k in range(1, m + 1):
        w.append(-w[-1] * ((d - k + 1) / k))
    return np.array(w)

In [8]:
d = 0.845
m = 100  # truncation order

w = frac_weights_1(d, m)
print(np.nansum(w))

w = frac_weights_2(d, m)
print(np.nansum(w))

w = frac_weights_3(d, m)
print(np.nansum(w))

w = frac_weights_4(d, m)
print(np.nansum(w))

w = frac_weights_5(d, m)
print(np.nansum(w))

0.003395536381106888
0.003395536381106581
0.003395536381106833
0.003395536381106833
0.003395536381106833


# Speed
The `numba` (list) version of the iterative estimation (formula 5) is by far the fastest

In [9]:
%timeit w = frac_weights_1(d, m)
%timeit w = frac_weights_2(d, m)
%timeit w = frac_weights_3(d, m)
%timeit w = frac_weights_4(d, m)
%timeit w = frac_weights_5(d, m)

630 µs ± 39.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.14 ms ± 69.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
115 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
54 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.4 µs ± 716 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Memory

In [10]:
%memit w = frac_weights_1(d, m)
%memit w = frac_weights_2(d, m)
%memit w = frac_weights_3(d, m)
%memit w = frac_weights_4(d, m)
%memit w = frac_weights_5(d, m)

peak memory: 131.54 MiB, increment: -0.22 MiB
peak memory: 131.54 MiB, increment: 0.00 MiB
peak memory: 131.54 MiB, increment: 0.00 MiB
peak memory: 131.54 MiB, increment: 0.00 MiB
peak memory: 131.54 MiB, increment: 0.00 MiB


# References
* Baliarsingh, P., 2013. Some new difference sequence spaces of fractional order and their dual spaces. Applied Mathematics and Computation 219, 9737–9742. https://doi.org/10.1016/j.amc.2013.03.073
* Jensen, A.N., Nielsen, M.Ø., 2014. A Fast Fractional Difference Algorithm. Journal of Time Series Analysis 35, 428–436. https://doi.org/10.1111/jtsa.12074
* Prado, M.L. de, 2018. Advances in Financial Machine Learning, 1st ed. Wiley.
* Schoffer, O., n.d. Modellierung von Kapitalmarktrenditen mittels asymmetrischer GARCH-Modelle. Universitaet Dortmund.
